In [28]:
import soundfile as sf

from transformers import Qwen2_5OmniThinkerForConditionalGeneration, Qwen2_5OmniProcessor
from qwen_omni_utils import process_mm_info

In [29]:
model_id = "Qwen/Qwen2.5-Omni-3B"
model = Qwen2_5OmniThinkerForConditionalGeneration.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
processor = Qwen2_5OmniProcessor.from_pretrained(model_id)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  6.03it/s]


In [30]:
conversation = [
    # {
    #     "role": "system",
    #     "content": [
    #         {"type": "text", "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."}
    #     ],
    # },
    {
        "role": "user",
        "content": [
            {"type": "audio", "audio": "data/datasets/LibriSpeech/LibriSpeech/test-clean/61/70968/61-70968-0000.flac"},
        ],
    },
]

In [31]:
USE_AUDIO_IN_VIDEO = True

# Preparation for inference
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
print("Text input for the model:")
print(text)
audios, images, videos = process_mm_info(conversation, use_audio_in_video=USE_AUDIO_IN_VIDEO)

print("Audio, Image, and Video inputs for the model:")
print("Audios:")
print(audios);
print(audios[0].shape)

print("Images:")
print(images)

print("Videos:")
print(videos)

Text input for the model:
['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|audio_bos|><|AUDIO|><|audio_eos|><|im_end|>\n<|im_start|>assistant\n']
Audio, Image, and Video inputs for the model:
Audios:
[array([-9.3078613e-03, -9.6130371e-03, -9.0637207e-03, ...,
        4.1198730e-03,  2.3803711e-03, -6.1035156e-05], dtype=float32)]
(78480,)
Images:
None
Videos:
None


In [41]:
inputs = processor(text=text, audio=audios, images=images, videos=videos, return_tensors="pt", padding=True, use_audio_in_video=USE_AUDIO_IN_VIDEO)
print("Processed inputs for the model:")
cnt = 0
for i in inputs.feature_attention_mask[0]:
	if i == 1:
		cnt += 1

inputs = inputs.to(model.device).to(model.dtype)

print(cnt)
print(inputs.input_features.shape)

Processed inputs for the model:
491
torch.Size([1, 128, 30000])


In [33]:
audio_features = model.get_audio_features(
	inputs.input_features,
	feature_attention_mask=inputs.feature_attention_mask,
)
print(audio_features.shape)

torch.Size([123, 2048])


In [34]:
outputs = model(**inputs, output_hidden_states=True)

In [35]:
inputs_embeds = model.get_input_embeddings()(inputs.input_ids)
print(inputs_embeds.shape)

torch.Size([1, 144, 2048])


In [39]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(tokenizer.batch_decode(inputs.input_ids, skip_special_tokens=False))

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|audio_bos|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO|><|AUDIO

In [42]:
# Inference: Generation of the output text and audio
text_ids, audio = model.generate(**inputs, use_audio_in_video=USE_AUDIO_IN_VIDEO)

text = processor.batch_decode(text_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(text)
sf.write(
    "output.wav",
    audio.reshape(-1).detach().cpu().numpy(),
    samplerate=24000,
)

ValueError: not enough values to unpack (expected 2, got 1)

In [14]:
import datasets

In [17]:
# load openslr/librispeech_asr

librispeech_asr = datasets.load_dataset("openslr/librispeech_asr")

FSTimeoutError: 